In [1]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.time import SimultaneousActivation
from mesa.space import MultiGrid
from mesa.space import SingleGrid
from mesa.datacollection import DataCollector
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import Slider

In [2]:
import psutil
def kill_process_using_port(port):
    for proc in psutil.process_iter(['pid', 'name', 'connections']):
        for conn in proc.info['connections']:
            if conn.laddr.port == port:
                try:
                    print(f"Killing process {proc.info['pid']} ({proc.info['name']}) using port {port}")
                    proc.terminate()
                    proc.wait()
                    return
                except (psutil.NoSuchProcess, psutil.AccessDenied):
                    print(f"Could not terminate process {proc.info['pid']} ({proc.info['name']})")

port = 8523
kill_process_using_port(port)

In [3]:
# Define the state constants
SUSCEPTIBLE = 0
ASYMPTOMATIC = 1  # Infected but no symptoms yet (incubation period)
SYMPTOMATIC = 2   # Infected with symptoms
RECOVERED = 3


class Person(Agent):
    """An agent representing a single person in the grid."""
    def __init__(self, pos, model):
        super().__init__(pos, model)
        self.pos = pos
        self.state = SUSCEPTIBLE
        self.incubation_time = 0
        self.symptom_time = 0
        self.next_state = self.state

    def step(self):
        if self.state == ASYMPTOMATIC:
            neighbors = self.model.grid.get_neighbors(self.pos, moore=False, include_center=False)
            for neighbor in neighbors:
                if neighbor.state == SUSCEPTIBLE:
                    neighbor.next_state = ASYMPTOMATIC
                    neighbor.incubation_time = self.model.incubation_period
            self.incubation_time -= 1

            if self.incubation_time <= 0:
                self.next_state = SYMPTOMATIC
                self.symptom_time = self.model.symptom_period

        elif self.state == SYMPTOMATIC:
            self.symptom_time -= 1

            if self.symptom_time <= 0:
                self.next_state = RECOVERED

    def advance(self):
        self.state = self.next_state


In [4]:
class DiseaseModel(Model):
    """A model with some number of agents."""
    def __init__(self, width, height, density, initial_infected, incubation_period, symptom_period):
        super().__init__()
        self.num_agents = width * height
        self.grid = SingleGrid(width, height, torus=True)
        self.schedule = SimultaneousActivation(self)
        self.incubation_period = incubation_period
        self.symptom_period = symptom_period

        for content, (x, y) in self.grid.coord_iter():
            agent = Person((x, y), self)
            self.grid.place_agent(agent, (x, y))
            self.schedule.add(agent)

        self.running = True
        
        center_x, center_y = width // 2, height // 2
        center_agent = self.grid.get_cell_list_contents([(center_x, center_y)])[0]
        center_agent.state = ASYMPTOMATIC
        center_agent.incubation_time = incubation_period

    def step(self):
        self.schedule.step()

In [5]:
def agent_portrayal(agent):
    portrayal = {"Shape": "rect", "w": 1, "h": 1,"Filled": "True",
                 "Layer": 0}
    if agent.state == SUSCEPTIBLE:
        portrayal["Color"] = "white"
    elif agent.state == ASYMPTOMATIC:
        portrayal["Color"] = "pink"
    elif agent.state == SYMPTOMATIC:
        portrayal["Color"] = "red"
    elif agent.state == RECOVERED:
        portrayal["Color"] = "gray"
    return portrayal

grid = CanvasGrid(agent_portrayal, 25, 25, 500, 500)
# Now you can launch your server
server = ModularServer(DiseaseModel,
                       [grid],
                       "Disease Model ( Incubation Period)",
                       {"width": 25,
                        "height": 25,
                        "density": Slider("Initial infected", 0.8, 0.1, 1.0, 0.1),
                        "initial_infected": Slider("Infection rate", 1, 0, 10, 1),
                        "incubation_period": Slider("Incubation period", 2, 0, 10, 1),
                        "symptom_period": Slider('Symptom Period', 8, 1, 20, 1)
                       })
server.port = port
server.launch()


C:\Users\asust\AppData\Local\Temp\ipykernel_25480\58251572.py:13: UserWarning: Agent (0, 0) is being placed with
place_agent() despite already having the position (0, 0). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(agent, (x, y))
C:\Users\asust\AppData\Local\Temp\ipykernel_25480\58251572.py:13: UserWarning: Agent (0, 1) is being placed with
place_agent() despite already having the position (0, 1). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(agent, (x, y))
C:\Users\asust\AppData\Local\Temp\ipykernel_25480\58251572.py:13: UserWarning: Agent (0, 2) is being placed with
place_agent() despite already having the position (0, 2). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(agent, (x, y))
C:\Users\asust\AppData\Local\Temp\ipykernel_25

Interface starting at http://127.0.0.1:8523


AssertionError: 